In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import csv


In [2]:
d = pd.read_csv("data.csv")
df = pd.DataFrame(d)
df

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


In [3]:
new_df = pd.get_dummies(df)
new_df

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,Sex_F,Sex_M,ChestPainType_ASY,...,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ExerciseAngina_N,ExerciseAngina_Y,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,40,140,289,0,172,0.0,0,0,1,0,...,0,0,0,1,0,1,0,0,0,1
1,49,160,180,0,156,1.0,1,1,0,0,...,1,0,0,1,0,1,0,0,1,0
2,37,130,283,0,98,0.0,0,0,1,0,...,0,0,0,0,1,1,0,0,0,1
3,48,138,214,0,108,1.5,1,1,0,1,...,0,0,0,1,0,0,1,0,1,0
4,54,150,195,0,122,0.0,0,0,1,0,...,1,0,0,1,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,110,264,0,132,1.2,1,0,1,0,...,0,1,0,1,0,1,0,0,1,0
914,68,144,193,1,141,3.4,1,0,1,1,...,0,0,0,1,0,1,0,0,1,0
915,57,130,131,0,115,1.2,1,0,1,1,...,0,0,0,1,0,0,1,0,1,0
916,57,130,236,0,174,0.0,1,1,0,0,...,0,0,1,0,0,1,0,0,1,0


In [4]:
# this is equivalent to the training portion of the tensors

y = new_df["HeartDisease"]
x = new_df.drop(columns=["HeartDisease"])

X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=78)

In [5]:
Scaler = StandardScaler()
X_Scaler = Scaler.fit(X_train)
X_train_scaled = X_Scaler.transform(X_train)
X_test_scaled = X_Scaler.transform(X_test)

In [6]:
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    #keras tuner will decide between the activation functions
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # decides number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=20)) #input dimension is just the feautures

    #   decides number of hidden layers/neurons in other layers (between 1 and 6)
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [7]:
import keras_tuner as kt

tuner_1 = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=10,
    hyperband_iterations=2)


In [8]:
tuner_1.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 00m 03s]
val_accuracy: 0.782608687877655

Best val_accuracy So Far: 0.834782600402832
Total elapsed time: 00h 01m 52s


In [9]:
best_hyper = tuner_1.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 7,
 'num_layers': 3,
 'units_0': 5,
 'units_1': 3,
 'units_2': 7,
 'units_3': 1,
 'units_4': 7,
 'units_5': 7,
 'tuner/epochs': 10,
 'tuner/initial_epoch': 4,
 'tuner/bracket': 2,
 'tuner/round': 2,
 'tuner/trial_id': '0042'}

In [10]:
nn = tf.keras.models.Sequential()

nn.add(tf.keras.layers.Dense(units=5, activation="tanh", input_dim=20))

nn.add(tf.keras.layers.Dense(units=9, activation="tanh"))

nn.add(tf.keras.layers.Dense(units=1, activation="tanh"))

nn.add(tf.keras.layers.Dense(units=7, activation="tanh"))

nn.add(tf.keras.layers.Dense(units=9, activation="tanh"))

nn.add(tf.keras.layers.Dense(units=5, activation="tanh"))

nn.add(tf.keras.layers.Dense(units=1, activation="tanh"))

nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 5)                 105       
                                                                 
 dense_9 (Dense)             (None, 9)                 54        
                                                                 
 dense_10 (Dense)            (None, 1)                 10        
                                                                 
 dense_11 (Dense)            (None, 7)                 14        
                                                                 
 dense_12 (Dense)            (None, 9)                 72        
                                                                 
 dense_13 (Dense)            (None, 5)                 50        
                                                                 
 dense_14 (Dense)            (None, 1)                

In [11]:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [12]:
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
22/22 [==============================] - 1s 2ms/step - loss: 3.4579 - accuracy: 0.4448
Epoch 2/100
22/22 [==============================] - 0s 2ms/step - loss: 0.9280 - accuracy: 0.4448
Epoch 3/100
22/22 [==============================] - 0s 2ms/step - loss: 0.7609 - accuracy: 0.4448
Epoch 4/100
22/22 [==============================] - 0s 1ms/step - loss: 0.7076 - accuracy: 0.4448
Epoch 5/100
22/22 [==============================] - 0s 2ms/step - loss: 0.6687 - accuracy: 0.5683
Epoch 6/100
22/22 [==============================] - 0s 2ms/step - loss: 0.6391 - accuracy: 0.6642
Epoch 7/100
22/22 [==============================] - 0s 2ms/step - loss: 0.6133 - accuracy: 0.7078
Epoch 8/100
22/22 [==============================] - 0s 2ms/step - loss: 0.5915 - accuracy: 0.7180
Epoch 9/100
22/22 [==============================] - 0s 1ms/step - loss: 0.5700 - accuracy: 0.7340
Epoch 10/100
22/22 [==============================] - 0s 2ms/step - loss: 0.5486 - accuracy: 0.7384
Epoch 11/

22/22 [==============================] - 0s 2ms/step - loss: 0.2957 - accuracy: 0.8939
Epoch 84/100
22/22 [==============================] - 0s 2ms/step - loss: 0.2966 - accuracy: 0.8924
Epoch 85/100
22/22 [==============================] - 0s 2ms/step - loss: 0.2949 - accuracy: 0.8953
Epoch 86/100
22/22 [==============================] - 0s 2ms/step - loss: 0.2943 - accuracy: 0.8924
Epoch 87/100
22/22 [==============================] - 0s 1ms/step - loss: 0.2937 - accuracy: 0.8939
Epoch 88/100
22/22 [==============================] - 0s 1ms/step - loss: 0.3687 - accuracy: 0.8895
Epoch 89/100
22/22 [==============================] - 0s 1ms/step - loss: 0.6864 - accuracy: 0.5552
Epoch 90/100
22/22 [==============================] - 0s 1ms/step - loss: 0.7422 - accuracy: 0.5552
Epoch 91/100
22/22 [==============================] - 0s 1ms/step - loss: 0.7036 - accuracy: 0.5552
Epoch 92/100
22/22 [==============================] - 0s 2ms/step - loss: 0.6604 - accuracy: 0.5552
Epoch 93/100


In [13]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8/8 - 0s - loss: 0.4510 - accuracy: 0.8130 - 140ms/epoch - 17ms/step
Loss: 0.4510350823402405, Accuracy: 0.813043475151062
